# Tracking Hyperparameter Tuning with MLflow

In [12]:
import mlflow

# The set_experiment API creates a new experiment if it doesn't exist.
mlflow.set_experiment("Hyperparameter Tuning Experiment")
mlflow.set_tracking_uri("http://localhost:5002")

2025/11/17 08:14:28 INFO mlflow.tracking.fluent: Experiment with name 'Hyperparameter Tuning Experiment' does not exist. Creating a new experiment.


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)

In [14]:
X.shape, y.shape

((20640, 8), (20640,))

## Define the objective function

In [15]:
import optuna
import sklearn

# Create the trial to have each run for childs
def objective(trial):
    # Setting nested=True will create a child run under the parent run.
    with mlflow.start_run(nested=True, run_name=f"trial_{trial.number}") as child_run:
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32)
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 50, 300, step=10)
        rf_max_features = trial.suggest_float("rf_max_features", 0.2, 1.0)
        params = {
            "max_depth": rf_max_depth,
            "n_estimators": rf_n_estimators,
            "max_features": rf_max_features,
        }
        # Log current trial's parameters
        mlflow.log_params(params)

        regressor_obj = sklearn.ensemble.RandomForestRegressor(**params)
        regressor_obj.fit(X_train, y_train)

        y_pred = regressor_obj.predict(X_val)
        error = sklearn.metrics.mean_squared_error(y_val, y_pred)
        # Log current trial's error metric
        mlflow.log_metrics({"error": error})

        # Log the model file
        mlflow.sklearn.log_model(regressor_obj, name="model")
        # Make it easy to retrieve the best-performing child run later
        trial.set_user_attr("run_id", child_run.info.run_id)
        return error

## Run the hyperparameter tuning study

In [16]:
# Create a parent run that contains all child runs for different trials
with mlflow.start_run(run_name="study") as run:
    # Log the experiment settings
    n_trials = 30
    mlflow.log_param("n_trials", n_trials)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)

    # Log the best trial and its run ID
    mlflow.log_params(study.best_trial.params)
    mlflow.log_metrics({"best_error": study.best_value})
    if best_run_id := study.best_trial.user_attrs.get("run_id"):
        mlflow.log_param("best_child_run_id", best_run_id)

[I 2025-11-17 08:14:40,180] A new study created in memory with name: no-name-41e6bbbb-ed52-411b-8ac4-0e58fb48773d
2025/11/17 08:14:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-11-17 08:14:44,745] Trial 0 finished with value: 0.4578706346615976 and parameters: {'rf_max_depth': 5, 'rf_n_estimators': 130, 'rf_max_features': 0.8699694067662267}. Best is trial 0 with value: 0.4578706346615976.


🏃 View run trial_0 at: http://localhost:5002/#/experiments/434642877658932656/runs/6536ecc71ac7467fb655aa7b8ea4b186
🧪 View experiment at: http://localhost:5002/#/experiments/434642877658932656


2025/11/17 08:14:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-11-17 08:15:13,176] Trial 1 finished with value: 0.26544167095345444 and parameters: {'rf_max_depth': 27, 'rf_n_estimators': 150, 'rf_max_features': 0.8902874620836059}. Best is trial 1 with value: 0.26544167095345444.


🏃 View run trial_1 at: http://localhost:5002/#/experiments/434642877658932656/runs/7df7022bdd3c4190acddd627b0a7115c
🧪 View experiment at: http://localhost:5002/#/experiments/434642877658932656


2025/11/17 08:15:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-11-17 08:15:27,526] Trial 2 finished with value: 0.24595368586452276 and parameters: {'rf_max_depth': 29, 'rf_n_estimators': 120, 'rf_max_features': 0.33665307685889734}. Best is trial 2 with value: 0.24595368586452276.


...

🏃 View run trial_27 at: http://localhost:5002/#/experiments/434642877658932656/runs/2440f066b06a4964941500ab6032fb57
🧪 View experiment at: http://localhost:5002/#/experiments/434642877658932656


2025/11/17 08:29:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-11-17 08:29:37,055] Trial 28 finished with value: 0.24663517941769456 and parameters: {'rf_max_depth': 18, 'rf_n_estimators': 100, 'rf_max_features': 0.4352881173244638}. Best is trial 22 with value: 0.24334859668150072.


🏃 View run trial_28 at: http://localhost:5002/#/experiments/434642877658932656/runs/39b96154ab5e4ef2b3eca04395c708cd
🧪 View experiment at: http://localhost:5002/#/experiments/434642877658932656


2025/11/17 08:29:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-11-17 08:29:41,364] Trial 29 finished with value: 0.3043112602075489 and parameters: {'rf_max_depth': 10, 'rf_n_estimators': 160, 'rf_max_features': 0.2880452898020235}. Best is trial 22 with value: 0.24334859668150072.


🏃 View run trial_29 at: http://localhost:5002/#/experiments/434642877658932656/runs/26da371cecc441edbc06f31d74af12cb
🧪 View experiment at: http://localhost:5002/#/experiments/434642877658932656
🏃 View run study at: http://localhost:5002/#/experiments/434642877658932656/runs/d2e008483ea24c4f882dff35c148df01
🧪 View experiment at: http://localhost:5002/#/experiments/434642877658932656


In [19]:
# Register the best model using the model URI
mlflow.register_model(
    model_uri="runs:/598e0bbe725c4f24ad61af0bbbbcab6c/model",
    name="housing-price-predictor-new",
)

# > Successfully registered model 'housing-price-predictor'.
# > Created version '1' of model 'housing-price-predictor'.

Successfully registered model 'housing-price-predictor-new'.
2025/11/17 09:02:18 WARNING mlflow.tracking._model_registry.fluent: Run with id 598e0bbe725c4f24ad61af0bbbbcab6c has no artifacts at artifact path 'model', registering model based on models:/m-7ac0f68d04dc48cb9662c13a40666c5a instead
2025/11/17 09:02:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: housing-price-predictor-new, version 1
Created version '1' of model 'housing-price-predictor-new'.


<ModelVersion: aliases=[], creation_timestamp=1763357538796, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1763357538796, metrics=None, model_id=None, name='housing-price-predictor-new', params=None, run_id='598e0bbe725c4f24ad61af0bbbbcab6c', run_link='', source='models:/m-7ac0f68d04dc48cb9662c13a40666c5a', status='READY', status_message=None, tags={}, user_id='', version='1'>